In [224]:
from sklearn import svm, preprocessing, metrics
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

In [629]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import random

In [3]:
#a = all_data[0]
# a.head()

In [4]:
# a.columns

In [5]:
def AU(n):
    # return AU column name
    return f'AU{str(n).zfill(2)}_r'

In [6]:
# ex = all_data[0]
# # for x in range(ex['milis'].max):
# #     print(i)

# x = 1
# windows = []
# while x + 3000 < max(ex['milis']):
#     windows.append((x, x + 3000))
#     x += 3010

# print(windows)
# len(windows)

In [234]:
# define useful functions

count = 0
RANGE = 1500
def closest_frame(df, f):
    f_near = f
    for x in range(50):
        while len(face_data.loc[face_data['frame'] == f_near, 'milis'].values) == 0:
            f_near -= 1
#     print("number of frames retraced: " + str(f - f_near))
    return f_near

def df_near(df, frame, ms_range=RANGE):
#     global count
#     count += len(frames_list)
    if frame is None:
        return None
    ms = df.loc[df['frame'] == closest_frame(df,frame), 'milis'].values[0]
    locs = df.loc[(df['milis'] >= ms - ms_range) & (df['milis'] <= ms + ms_range)]
#     locs = df.loc[(df['milis'] >= ms - ms_range) & (df['milis'] <= ms)]
    return locs

def df_near_tuple(df, milis, ms_range=RANGE):
    locs = df.loc[(df['milis'] >= milis[0]) & (df['milis'] <= milis[1])]
    return locs

def max_df_near(df, frames_list, ms_range=RANGE):
    d = df_near(df, frames_list)
    if d is None:
        return None
    return pd.DataFrame(d.max()).transpose()

def min_df_near(df, frames_list, ms_range=RANGE):
    d = df_near(df, frames_list)
    if d is None:
        return None
    return pd.DataFrame(d.min()).transpose()

def max_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.max()).transpose()

def min_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.min()).transpose()

def mean_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.mean()).transpose()

def sd_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.std()).transpose()

In [235]:
def overlap(a,b):
    return a[0] <= b[0] <= a[1] or b[0] <= a[0] <= b[1]

In [236]:
# gather data

# choose which action units to look at
AUs_to_look_at = [9, 14, 15, 17, 20, 23, 45]

def get_pid(filename):
    """ get pid from dirname """
    return filename.split('_')[0][1:]
def get_mode(filename):
    """ get mode from dirname """
    return int(file.split('_')[2][1])

up_data_max = []
down_data_max = []
up_data_min = []
down_data_min = []
all_data = []

up_data_sd = []
down_data_sd = []

up_data_mean = []
down_data_mean = []

neutral_data_max = []
neutral_data_min = []
neutral_data_mean = []
neutral_data_sd = []

all_data_max = []
all_data_min = []
all_data_mean = []
all_data_sd = []

PATH = "/Users/yoony/Desktop/si_research/face_analysis/implicit_feedback_si_faceanalysis/processed"
csvs = []
for file in sorted(os.listdir(PATH)):
    if file[0] == '.':
        continue
    print(file)
#     csvs.append(pd.read_csv(os.path.join(PATH + "/" + file, 'webcam/aggregated_webcam.csv')))
    data = pd.read_csv(os.path.join(PATH + "/" + file, 'webcam/aggregated_webcam.csv'))
    face_data = data.drop(columns=list(data.columns.values[2:678]))
    face_data = face_data.drop(columns=list(face_data.columns.values[19:]))
    face_data = face_data[(['frame', 'milis'] + [AU(x) for x in AUs_to_look_at])]
    
    signals = pd.read_csv('../my_buckets.csv')[['player_id', 'mode', 'signal_type', 'frame_number', 'code']]
    s1 = signals.loc[(signals['player_id'] == get_pid(file)) & (signals['mode'] == get_mode(file))]
    s_up = s1.loc[s1['signal_type'] == 'up'].frame_number.values
    s_down = s1.loc[s1['signal_type'] == 'down'].frame_number.values
    
    covered_windows = []
    
    for x in s_up:
        up_locs = df_near(face_data, x)
        up_data_max.append(max_df(up_locs))
        up_data_min.append(min_df(up_locs))
        up_data_sd.append(sd_df(up_locs))
        up_data_mean.append(mean_df(up_locs))
        
        all_data_max.append(max_df(up_locs))
        all_data_min.append(min_df(up_locs))
        all_data_sd.append(sd_df(up_locs))
        all_data_mean.append(mean_df(up_locs))
        
        covered_windows.append((min(up_locs['milis']), max(up_locs['milis'])))
        
    for x in s_down:
        down_locs = df_near(face_data, x)
        down_data_max.append(max_df(down_locs))
        down_data_min.append(min_df(down_locs))
        down_data_sd.append(sd_df(down_locs))
        down_data_mean.append(mean_df(down_locs))
        
        all_data_max.append(max_df(down_locs))
        all_data_min.append(min_df(down_locs))
        all_data_sd.append(sd_df(down_locs))
        all_data_mean.append(mean_df(down_locs))
        
        covered_windows.append((min(down_locs['milis']), max(down_locs['milis'])))
    
    windows = []
    while x + RANGE/2 < max(face_data['milis']):
        w = (x, x+(RANGE/2))
        clash = False
        for c in covered_windows:
            if overlap(w,c):
                clash = True
                x = c[1] + 100
        if not clash:
            windows.append(w)
            x += 3100
    print("stat" + str(max(face_data['milis'] / 3000)))
    print(covered_windows)
    print(len(windows))
    for w in windows:
        neutral_locs = df_near_tuple(face_data, w)
        neutral_data_max.append(max_df(neutral_locs))
        neutral_data_min.append(min_df(neutral_locs))
        neutral_data_sd.append(sd_df(neutral_locs))
        neutral_data_mean.append(mean_df(neutral_locs))
        
        all_data_max.append(max_df(neutral_locs))
        all_data_min.append(min_df(neutral_locs))
        all_data_sd.append(sd_df(neutral_locs))
        all_data_mean.append(mean_df(neutral_locs))

        
    all_data.append(face_data)
    
    

P5ee6d2494fd04045b988a12d_voff_m1_g1_t2021_7_10_0_47
stat20.664
[(11174, 14112), (16438, 19372), (20238, 23168), (27508, 30439), (35444, 38378), (50580, 53518), (57116, 60054), (60387, 61992)]
14
P5ee6d2494fd04045b988a12d_voff_m2_g2_t2021_7_10_0_50
stat14.279
[(4533, 7464), (8267, 11200), (17270, 20205), (41037, 42837)]
10
P5ef33c05f4c5240413ef9ac2_voff_m1_g1_t2021_7_13_20_38
stat16.415666666666667
[(3906, 6810), (15126, 18030), (24761, 27665), (47399, 49247), (10045, 12947), (28655, 31561), (30306, 33210), (34398, 37302), (45024, 47928)]
10
P5ef33c05f4c5240413ef9ac2_voff_m2_g2_t2021_7_13_20_34
stat14.203333333333333
[(4330, 7234), (10269, 13174), (15088, 17992), (22347, 25251), (28090, 30995), (33766, 36670)]
9
P6081c511ded5945d9b70801e_voff_m1_g1_t2021_7_13_21_6
stat18.284333333333333
[(2581, 5485), (9712, 12612), (14792, 17697), (23503, 26407), (37231, 40136)]
14
P6081c511ded5945d9b70801e_voff_m2_g2_t2021_7_13_21_11
stat18.292666666666666
[(16006, 18909), (19373, 22275), (31119, 340

In [237]:
print(len(neutral_data_max))
print(len(up_data_max))
print(len(down_data_max))

153
64
22


In [238]:
def rename_columns(df, tag):
    a = df.copy()
    a.columns = list(map(lambda x: x + tag, a.columns))
    return a

def merge_dfs(dfs):
    a = dfs[0]
    for d in dfs[1:]:
        a = a.join(d, how='outer')
    return a
        

In [239]:
u1 = rename_columns(pd.concat(up_data_max).drop(columns=['frame', 'milis']), '_max').reset_index(drop=True)
u2 = rename_columns(pd.concat(up_data_min).drop(columns=['frame', 'milis']), '_min').reset_index(drop=True)
u3 = rename_columns(pd.concat(up_data_mean).drop(columns=['frame', 'milis']), '_mean').reset_index(drop=True)
u4 = rename_columns(pd.concat(up_data_sd).drop(columns=['frame', 'milis']), '_sd').reset_index(drop=True)

In [450]:
# ups = merge_dfs([u1, u2, u3, u4])
ups = merge_dfs([u1, u2, u3, u4])
ups['type'] = 'up'

# ups['type'] = 'signal'

In [451]:
d1 = rename_columns(pd.concat(down_data_max).drop(columns=['frame', 'milis']), '_max').reset_index(drop=True)
d2 = rename_columns(pd.concat(down_data_min).drop(columns=['frame', 'milis']), '_min').reset_index(drop=True)
d3 = rename_columns(pd.concat(down_data_mean).drop(columns=['frame', 'milis']), '_mean').reset_index(drop=True)
d4 = rename_columns(pd.concat(down_data_sd).drop(columns=['frame', 'milis']), '_sd').reset_index(drop=True)

In [452]:
# downs = merge_dfs([d1, d2, d3, d4])
downs = merge_dfs([d1, d2, d3, d4])
downs['type'] = 'down'
# downs['type'] = 'signal'

In [453]:
n1 = rename_columns(pd.concat(neutral_data_max).drop(columns=['frame', 'milis']), '_max').reset_index(drop=True)
n2 = rename_columns(pd.concat(neutral_data_min).drop(columns=['frame', 'milis']), '_min').reset_index(drop=True)
n3 = rename_columns(pd.concat(neutral_data_mean).drop(columns=['frame', 'milis']), '_mean').reset_index(drop=True)
n4 = rename_columns(pd.concat(neutral_data_sd).drop(columns=['frame', 'milis']), '_sd').reset_index(drop=True)

In [454]:
neutrals = merge_dfs([n1,n2,n3,n4])
neutrals['type'] = 'neutral'

In [455]:
means = pd.DataFrame(pd.concat(up_data_max).mean(), columns=['up_mean'])
means['down_mean'] = pd.concat(down_data_max).mean()
means['mean_diff'] = means['up_mean'] - means['down_mean']
means['neutral_mean'] = pd.concat(neutral_data_max).mean()
means['up_sd'] = pd.concat(up_data_sd).std()
means['down_sd'] = pd.concat(down_data_sd).std()
means['sd_diff'] = means['up_sd'] - means['down_sd']
means = means.drop(index=['frame', 'milis'])
# means['total'] = pd.concat(all_data).mean
# means.to_csv('face/au_data.csv')
means

,up_mean,down_mean,mean_diff,neutral_mean,up_sd,down_sd,sd_diff
AU09_r,0.594219,0.220455,0.373764,0.385862,0.102431,0.060722,0.041709
AU14_r,0.637188,0.318182,0.319006,0.353448,0.114204,0.079375,0.034830
AU15_r,0.501406,0.779545,-0.278139,0.550552,0.110572,0.072796,0.037776
AU17_r,0.830781,0.447727,0.383054,0.476069,0.174185,0.111115,0.063070
AU20_r,0.806563,1.017273,-0.210710,0.627034,0.094162,0.078059,0.016103
AU23_r,0.760156,0.474545,0.285611,0.437793,0.110847,0.071134,0.039714
AU45_r,0.642344,0.964545,-0.322202,0.413793,0.151040,0.190043,-0.039003


In [456]:
# check number of signals
len(up_data_max) + len(down_data_max) + len(neutral_data_max)

239

In [457]:
len(neutrals)

153

In [458]:
clf = svm.SVC(kernel='linear')

In [459]:
#clean up and finalize the data
signal_data = pd.concat([ups, downs, neutrals]).reset_index(drop=True)
signal_data = signal_data.dropna()
signal_types = signal_data['type'].values
signal_data = signal_data.drop(columns=['type'])

In [634]:
X = signal_data
y = signal_types

Counter(y)

Counter({'up': 64, 'down': 22, 'neutral': 145})

In [635]:
# # split up into testing and training
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=109)

In [636]:
# # normalize inputs
# normalizer = preprocessing.Normalizer()

# norm_train_X = normalizer.fit_transform(X_train)

# norm_test_X = normalizer.transform(X_test)


# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train_scaled = scaler.transform(X_train)

# scaler = preprocessing.StandardScaler().fit(X_test)
# X_test_scaled = scaler.transform(X_test)

In [637]:
def normalize(train, test):
    normalizer = preprocessing.Normalizer()
    return normalizer.fit_transform

In [638]:
# seed = 109 #random seed
# seed = random.randint(0,200)
# ros = RandomOverSampler(random_state=seed)
# smote = SMOTE(random_state=seed)
# adasyn = ADASYN(random_state=seed)

# under = RandomUnderSampler(random_state=seed)
# # X_resampled, y_resampled = ros.fit_resample(X, y)
# # X_resampled, y_resampled = adasyn.fit_resample(X, y)

# # X_resampled, y_resampled = X, y

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
# # X_train_resampled, y_train_resampled = X_train, y_train

# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
# # X_train_resampled, y_train_resampled = under.fit_resample(X_train_resampled, y_train_resampled)

# # normalize inputs
# normalizer = preprocessing.Normalizer()

# norm_X_train = normalizer.transform(X_train_resampled)

# norm_X_test = normalizer.transform(X_test)

# # X_train_resampled, y_train_resampled = smote.fit_resample(norm_X_train, y_train)
# # X_train_resampled, y_train_resampled = X_train, y_train

In [771]:
# normalize first

seed = 109 #random seed
# seed = random.randint(0,200)
ros = RandomOverSampler(random_state=seed)
smote = SMOTE(random_state=seed)
adasyn = ADASYN(random_state=seed)

under = RandomUnderSampler(random_state=seed)
# X_resampled, y_resampled = ros.fit_resample(X, y)
# X_resampled, y_resampled = adasyn.fit_resample(X, y)

# X_resampled, y_resampled = X, y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

# normalize inputs
normalizer = preprocessing.Normalizer()

norm_X_train = normalizer.fit_transform(X_train)

norm_X_test = normalizer.transform(X_test)

X_train_resampled, y_train_resampled = smote.fit_resample(norm_X_train, y_train)
# X_train_resampled, y_train_resampled = under.fit_resample(X_train_resampled, y_train_resampled)

# X_train_resampled, y_train_resampled = smote.fit_resample(norm_X_train, y_train)
# X_train_resampled, y_train_resampled = X_train, y_train

In [772]:
# from collections import Counter
# counter = Counter(y_train)
# print('original: ' + str(counter))
# print('resampled: ' + str(Counter(y_train_resampled)))

In [773]:
# fit data
clf.fit(X_train_resampled, y_train_resampled)

# predict
y_pred = clf.predict(norm_X_test)

print(f"down: {metrics.f1_score(y_test, y_pred, average='weighted', labels=['down'])}")
print(f"up: {metrics.f1_score(y_test, y_pred, average='weighted', labels=['up'])}")
print(f"neutral: {metrics.f1_score(y_test, y_pred, average='weighted', labels=['neutral'])}")

# metrics.f1_score(y_test, y_pred, average='weighted')

down: 0.36363636363636365
up: 0.2926829268292683
neutral: 0.5454545454545454


In [737]:
r = pd.DataFrame(columns=['test', 'pred'])
r['test'] = y_test
r['pred'] = y_pred
r

,test,pred
0,up,neutral
1,down,down
2,up,down
3,up,up
4,up,down
...,...,...
65,up,up
66,neutral,up
67,down,down
68,neutral,neutral
